In [ ]:
%run ResourceAssembly_func.ipynb

# Simulations for Fig. 5b (originally done in separate runs for different conditions and were merged)

In [ ]:
Ss = [3,5,7,10,12];
Rs= [1,3,5,10,20,30,50,75,100];
survs={};
growths={};

assumptions=a_default.copy() #Start with default parameters
assumptions['sampling']='Shuffle'; # parameter distribution: discrete uniform
assumptions['l'] = 0; assumptions['p_cf']=0; # no cross-feeding
assumptions['R0_food']=10; # resource concentration scale
assumptions['tau']=50; # 1/dilution rate scale
assumptions['regulation']='independent'; #
assumptions['response']='type I';
assumptions['norm']=0;
for i in np.arange(len(Ss)):
    for j in np.arange(len(Rs)):
        S = Ss[i]; R = Rs[j]; pairs = list(itertools.combinations(np.arange(S), 2)); npair = len(pairs);
        survs[(S,R)]= np.zeros( (assumptions['n_wells'], npair+1, S) ) ;
        growths[(S,R)]= np.zeros( (assumptions['n_wells'], S, R) ) ;
        assumptions['S']=S;
        assumptions['MA']=1*np.ones(R);
        #assumptions['R0_food']=np.ones(assumptions['n_wells']);
        #assumptions['M']=1;
        assumptions['SA']=assumptions['S']*assumptions['n_wells']*np.ones(1);
        for k in np.arange(10):
            init_state = MakeInitialState(assumptions)
            def dNdt(N,R,params):
                return MakeConsumerDynamics(assumptions)(N,R,params)
            def dRdt(N,R,params):
                return MakeResourceDynamics(assumptions)(N,R,params)
            dynamics = [dNdt,dRdt]

            #Make parameters
            params = MakeParams(assumptions);
            params['m']=1/assumptions['tau'];
            #dumi = np.random.uniform(0.1,1,size=R); dumi=dumi/np.sum(dumi);
            ## for cross-feeding: supply only the most complex resource.
            dumi = np.zeros(R); dumi[R-1]=1;
            params['R0'] = assumptions['R0_food']*dumi;
            surv, growth = BottomUp(init_state, dynamics, params)
            ## use BottomUp_propagate for LLM
            if(k==0):
                survs[(S,R)] = surv;
                growths[(S,R)] = growth;
            else:
                survs[(S,R)] =np.concatenate( [survs[(S,R)], surv], axis=0 )
                growths[(S,R)] =np.concatenate( [growths[(S,R)], growth], axis=0 )
            save_obj(survs, 'survs_test')
            save_obj(growths, 'growths_test')

## Bistable simulation

In [ ]:
S=3; R=3; nsim = 5000;

survs = np.zeros((nsim, 10, 3));
growths = np.zeros((nsim, 3, 3));
invyields = np.zeros((nsim, 3, 3));
d=0.02;

for k in tqdm(np.arange(nsim)):
    R = np.random.random(size=(3,3)); R = R/np.sum(R)*9;
    A = np.random.random(size=(3,3)); A = A/np.sum(A)*9;
    RA = R*A;
    survs[k] = BottomUp_b(d, R, RA, c0=[1,1,1], tmax=100000, nstep=100000);
    growths[k] = R;
    invyields[k] = A;

In [ ]:
save_obj(survs, '220815_survs_bistable')
save_obj(growths, '220815_growths_bistable')
save_obj(invyields, '220815_invyields_bistable')